In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import uuid
import logging
import getpass
import numpy as np
import cv2
import os
import gc
import time
import imageio
import random
import shutil
import copy
import glob
import json
import shutil
import pickle
import traceback
import zipfile
from collections import Counter
from pprint import pprint as print

In [ ]:
import cv2
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from tqdm import tqdm, tqdm_notebook
from PIL import Image, ImageDraw, ExifTags
from shapely.geometry import Polygon
import imgaug
import imgaug as ia
import imgaug.augmenters as iaa
import skimage
from skimage import io, color
import torch
from torch.utils.data import DataLoader

In [ ]:
def imshow(img, figsize=(16, 16)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img)

In [ ]:
from test_api import CLI, CVAT_API_V1, ResourceType

In [ ]:
class Args:
    
    def __init__(self, action="create"):
        
        self.action = action
        self.auth = ("", "")  # type tuple
        self.server_host = ""
        self.server_port = 8080
        self.https = False
        self.loglevel = 20

        self.name = "test"
        self.labels = [{"name":"text", "attributes": []}]  # type list
        self.overlap = 0
        self.segment_size = 0
        self.bug = ""
        self.resource_type = ResourceType(0)
        self.resources = ["/home/phan.huy.hoang/Pictures/test_img_cvat.zip"]
        self.annotation_path = "/home/phan.huy.hoang/cvat/utils/cli/instances_default.json"
        self.annotation_format = "COCO 1.0"

        # for git only
        self.completion_verification_period = 10
        self.dataset_repository_url = ""
        self.lfs = False

args = Args()
CVAT_URL = "http://{}:{}/".format(args.server_host, args.server_port)
API_URL = CVAT_URL + "api/v1/"
username, password = args.auth
username, password

In [ ]:
def get_cookies():
    login_json = {"username": username, "password": password}
    login_req = requests.post(API_URL + 'auth/login', json=login_json)
    cookies = login_req.cookies
    return cookies

cookies = get_cookies()

In [ ]:
req = requests.get("{}/api/v1/tasks/1043".format(CVAT_URL), cookies=cookies)
req.text

In [ ]:
task_ids = list(range(1043, 1047)) + list(range(1048, 1063))  # ignore task 1047
print(len(task_ids))

### Download cả ảnh + label

In [ ]:
saved_dir = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/downloaded_cropper/zip"

In [ ]:
url_fmt = "http://{}:8080/api/v1/tasks/{}/dataset?format=COCO%201.0&action=download"

In [ ]:
def download_file(req, task_id):
    with open(os.path.join(saved_dir, '{}.zip'.format(task_id)), 'wb') as f:
        for chunk in req.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("Task {}: download completed".format(task_id))

In [ ]:
for task_id in tqdm_notebook(task_ids):
    url = url_fmt.format(args.server_host, task_id)

    while True:
        req = requests.get(url, cookies=cookies)
        if req.status_code == 200:
            download_file(req, task_id)
            break

    time.sleep(10)
    
# unzip all zip files in the same root folder
for task_id in task_ids:
    zip_fp = os.path.join(saved_dir, "{}.zip".format(task_id))
    assert os.path.exists(zip_fp)
    
    task_dir = os.path.join(saved_dir, str(task_id))
    if not os.path.exists(task_dir):
        os.mkdir(task_dir)
        
    with zipfile.ZipFile(zip_fp, "r") as zip_ref:
        zip_ref.extractall(task_dir)

### Cắt ảnh theo perspective transform (4 góc)

In [ ]:
def get_center_point(c):
    x, y, w, h = c
    x_center = int(x + w / 2)
    y_center = int(y + h / 2)
    return x_center, y_center

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [ ]:
def remove_all_content(fd):
    shutil.rmtree(fd)
    os.mkdir(fd)

# remove_all_content(warped_dir)

In [ ]:
def check_existed_or_create(fd):
    if not os.path.exists(fd):
        os.mkdir(fd)
        
for type_idcard in ["cmnd", "cccd", "cccd_chip"]:
    first_dir = os.path.join(warped_dir, type_idcard)
    check_existed_or_create(first_dir)
    for mode in ["front", "back"]:
        second_dir = os.path.join(first_dir, mode)
        check_existed_or_create(second_dir)

In [ ]:
warped_dir = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/downloaded_cropper/warped"
assert os.path.exists(warped_dir)

In [ ]:
error_fps = []

def process_task(anno_fp, img_dir):
    # for 1 task
    with open(anno_fp, "r") as f:
        json_data = json.load(f)
    
    json_df = pd.DataFrame(json_data["annotations"])
    json_df = pd.concat([json_df.drop(['attributes'], axis=1), json_df['attributes'].apply(pd.Series)], axis=1)
    
    label_mapping = dict()
    for d in json_data["categories"]:
        label_mapping[d["name"]] = d["id"]
    
    for img_index, img_info in enumerate(tqdm_notebook(json_data["images"])):
        img_id = img_info["id"]
        img_fn = img_info["file_name"]
        img_fp = os.path.join(img_dir, img_fn)

        try:
            img_df = json_df[json_df["image_id"].isin([img_id])]
            pil_img = Image.open(img_fp)
            pil_img = pil_img.convert('RGB')
            img = np.array(pil_img)

            if len(img_df) == 4:
                is_front = False
            else:
                is_front = True
            # check nb rows == 4 or 5 (4/5 classes)
            # check 4 or 5 unique category ids (ignore wrong annotation)
            # idcard's type it not np.nan (annotated in top-left object)
            try:
                assert len(img_df) in [4, 5]
                if is_front:
                    assert len(img_df["category_id"].unique()) == 5
                else:
                    assert len(img_df["category_id"].unique()) == 4
                assert img_df[img_df["category_id"] == 1]["type"].values[0] is not np.nan
            except Exception:
    #             print("Error: {}".format(img_fp))
                error_fps.append(img_fp)
                continue

            is_valid = True
            for row_index, row in img_df.iterrows():
                c_id = row["category_id"]
                bbox = copy.deepcopy(row["bbox"])
                if c_id == label_mapping["top_left"]:
                    tl = bbox
                    type_idcard = row["type"]
                    if type_idcard is np.nan:
                        is_valid = False
                        break
                elif c_id == label_mapping["top_right"]:
                    tr = bbox
                elif c_id == label_mapping["bottom_left"]:
                    bl = bbox
                elif c_id == label_mapping["bottom_right"]:
                    br = bbox
            if not is_valid:
                continue

            if type_idcard not in ["cmnd", "cccd", "cccd_chip"]:
                error_fps.append(img_fp)
                continue

            pts = np.array([
                get_center_point(tl),
                get_center_point(tr),
                get_center_point(br),
                get_center_point(bl)
            ])
            pts = order_points(pts)
            warped_img = four_point_transform(img, pts)

            if is_front:
                sub_dir = "front"
            else:
                sub_dir = "back"
            warped_img_fp = os.path.join(warped_dir, type_idcard, sub_dir, "warped_{}".format(img_fn))

            # rotate image
            h, w, _ = warped_img.shape
            if h > w:
                # check rotated 90 degree image
                x1, y1 = get_center_point(tl)
                x3, y3 = get_center_point(br)
                if x1 > x3:
                    # rotate -90 degree
                    warped_img = cv2.rotate(warped_img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                else:
                    # rotate 90 degree
                    warped_img = cv2.rotate(warped_img, cv2.ROTATE_90_CLOCKWISE)
            else:
                # check reverse image
                y_center_top = (get_center_point(tl)[1] + get_center_point(tr)[1]) / 2 
                y_center_bottom = (get_center_point(bl)[1] + get_center_point(br)[1]) / 2 
                if y_center_top > y_center_bottom:
                    warped_img = cv2.rotate(warped_img, cv2.ROTATE_180)

            del tl, tr, br, bl
        #     print("{} - {}".format(img_fn, is_convert_rgb))
#             print("{} - {}".format(img.shape, warped_img.shape))
            imageio.imwrite(warped_img_fp, warped_img)
        except Exception:
            error_fps.append(img_fp)
            traceback.print_exc()
            continue

In [ ]:
for task_id in tqdm_notebook(task_ids):
    task_dir = os.path.join(saved_dir, str(task_id))
    img_dir = os.path.join(task_dir, "images")
    anno_fp = os.path.join(task_dir, "annotations", "instances_default.json")
    assert os.path.exists(task_dir)
    assert os.path.exists(anno_fp)
    assert os.path.exists(img_dir)
    
    process_task(anno_fp, img_dir)

In [ ]:
len(error_fps)